In [16]:
import os
import json
import tempfile
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

import mlflow
import mlflow.sklearn

In [17]:
df = pd.read_csv('final_df.csv')

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
df.head()

,builtup_area,rooms,bathrooms,balcony,facing,gas_pipline,gated_community,swimming_pool,gym,intercom,power_backup,garden,sports,current_floor,total_floor,lease_type,covered_parking,open_parking,school,bus_stop,railway,mall,metro_station,hospital,restaurant,rent,latitude,longitude,furnish_encoded
0,550.0,1.0,2.0,0.0,West,0,1,1,1,1,1,1,1,25.0,34.0,Family,1.0,0.0,0,0,1,0,1,1,0,41000.0,19.1627,72.8398,1
1,954.0,2.0,2.0,0.0,East,0,1,1,1,1,1,1,1,12.0,18.0,Bachelor Company Family,1.0,0.0,1,0,0,0,0,1,1,70000.0,19.0160,72.8760,0
2,550.0,1.0,2.0,0.0,West,0,0,0,0,0,0,0,0,1.0,4.0,Bachelor Company Family,0.0,0.0,0,0,0,0,1,1,0,50000.0,19.1300,72.8250,2
3,1500.0,3.0,3.0,0.0,West,0,0,0,0,1,1,0,0,5.0,7.0,Family,1.0,0.0,1,0,0,0,0,1,1,240000.0,19.0560,72.8300,1
4,2800.0,3.0,3.0,0.0,none,1,1,1,1,1,1,1,1,6.0,56.0,Bachelor Company,1.0,1.0,1,0,0,0,0,1,1,350000.0,19.0160,72.8660,1


In [20]:
df.facing.value_counts()

facing
East          7848
none          7209
North-East    3081
West          2924
North         1509
North-West     833
South-West     464
South          399
South-East     393
Name: count, dtype: int64

In [21]:
RANDOM_STATE = 42
TEST_SIZE = 0.2
CV = 3
N_JOBS = -1
SCORING = 'r2'

EXPERIMENT_NAMES = [
    "regression_no_scaler",
    "regression_with_scaler",
    "regression_with_scaler_and_pca"
]

In [22]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def ensure_experiment(name):
    client = mlflow.tracking.MlflowClient()
    existing = client.get_experiment_by_name(name)
    if existing is not None:
        return existing.experiment_id
    else:
        return client.create_experiment(name)

def log_grid_results_to_csv(grid, path):
    df = pd.DataFrame(grid.cv_results_)
    df.to_csv(path, index=False)
    return path


In [23]:
def load_data(df):
    data = df
    X = data.drop(columns=['rent'])
    y = data.rent
    return X, y


In [24]:
def get_models_and_grids():
    models = {}

    models['LinearRegression'] = (
        LinearRegression(),
        {'model__fit_intercept': [True, False]}
    )

    models['Ridge'] = (
        Ridge(random_state=RANDOM_STATE),
        {'model__alpha': [0.01, 0.1, 1.0, 10.0]}
    )

    models['Lasso'] = (
        Lasso(random_state=RANDOM_STATE, max_iter=5000),
        {'model__alpha': [0.001, 0.01, 0.1, 1.0]}
    )

    models['ElasticNet'] = (
        ElasticNet(random_state=RANDOM_STATE, max_iter=5000),
        {'model__alpha': [0.001, 0.01, 0.1, 1.0], 'model__l1_ratio': [0.2, 0.5, 0.8]}
    )

    models['DecisionTree'] = (
        DecisionTreeRegressor(random_state=RANDOM_STATE),
        {'model__max_depth': [None, 5, 10], 'model__min_samples_split': [2, 5]}
    )

    models['RandomForest'] = (
        RandomForestRegressor(random_state=RANDOM_STATE),
        {'model__n_estimators': [50, 100], 'model__max_depth': [None, 10]}
    )

    models['GradientBoosting'] = (
        GradientBoostingRegressor(random_state=RANDOM_STATE),
        {'model__n_estimators': [50, 100], 'model__learning_rate': [0.05, 0.1]}
    )

    models['SVR'] = (
        SVR(),
        {'model__kernel': ['rbf', 'linear'], 'model__C': [0.1, 1.0, 10.0]}
    )

    models['KNeighbors'] = (
        KNeighborsRegressor(),
        {'model__n_neighbors': [3, 5, 9], 'model__weights': ['uniform', 'distance']}
    )

    # 🚀 XGBoost
    models['XGBoost'] = (
        XGBRegressor(objective='reg:squarederror', random_state=RANDOM_STATE, verbosity=0, n_jobs=-1),
        {
            'model__n_estimators': [50, 100],
            'model__max_depth': [3, 5, 7],
            'model__learning_rate': [0.05, 0.1],
            'model__subsample': [0.8, 1.0],
            'model__colsample_bytree': [0.8, 1.0]
        }
    )

    return models

In [25]:
def run_experiments(df):
    X, y = load_data(df)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )

    # Detect categorical and numeric columns automatically
    cat_cols = X_train.select_dtypes(include=["object", "category"]).columns
    num_cols = X_train.select_dtypes(exclude=["object", "category"]).columns

    models_and_grids = get_models_and_grids()

    for exp_name in EXPERIMENT_NAMES:
        exp_id = ensure_experiment(exp_name)
        mlflow.set_experiment(exp_name)
        print(f"\n=== Running experiment: {exp_name} ===")

        # Define preprocessing per experiment
        if exp_name == "regression_no_scaler":
            numeric_transformer = "passthrough"
        elif exp_name == "regression_with_scaler":
            numeric_transformer = StandardScaler()
        else:  # with scaler + PCA
            numeric_transformer = Pipeline(
                [("scaler", StandardScaler()), ("pca", PCA())]
            )

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", numeric_transformer, num_cols),
                ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ]
        )

        for model_name, (estimator, grid) in models_and_grids.items():
            pipe = Pipeline(steps=[("preprocessor", preprocessor), ("model", estimator)])

            # If PCA present in pipeline, add grid for pca__n_components
            if exp_name == "regression_with_scaler_and_pca":
                grid = grid.copy()
                grid["preprocessor__num__pca__n_components"] = [0.95, 0.99, min(10, X_train.shape[1])]

            with mlflow.start_run(run_name=f"{model_name}_{datetime.now():%Y%m%d_%H%M%S}"):
                print(f"  ▶ Model: {model_name}")

                mlflow.log_param("experiment", exp_name)
                mlflow.log_param("model_name", model_name)

                gs = GridSearchCV(
                    pipe,
                    param_grid=grid,
                    cv=CV,
                    scoring=SCORING,
                    n_jobs=N_JOBS,
                    verbose=1,
                    return_train_score=False,
                )

                gs.fit(X_train, y_train)

                best = gs.best_estimator_
                y_pred = best.predict(X_test)
                r2 = r2_score(y_test, y_pred)
                test_rmse = rmse(y_test, y_pred)

                # Log results
                for k, v in gs.best_params_.items():
                    mlflow.log_param(k, v)
                mlflow.log_metric("r2_test", r2)
                mlflow.log_metric("rmse_test", test_rmse)

                # Log grid results and model
                tmpdir = tempfile.mkdtemp()
                grid_path = os.path.join(tmpdir, f"grid_results_{model_name}.csv")
                log_grid_results_to_csv(gs, grid_path)
                mlflow.log_artifact(grid_path, artifact_path="grid_search")

                mlflow.sklearn.log_model(best, artifact_path="model")

                print(f"    ✓ R2={r2:.4f}, RMSE={test_rmse:.4f}")

    print("\n🎯 All experiments complete! Launch MLflow UI with:")
    print("   mlflow ui --port 5000")

In [26]:
run_experiments(df)


=== Running experiment: regression_no_scaler ===
  ▶ Model: LinearRegression
Fitting 3 folds for each of 2 candidates, totalling 6 fits


2025/11/12 09:56:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 09:56:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ R2=0.7364, RMSE=56015.9546
  ▶ Model: Ridge
Fitting 3 folds for each of 4 candidates, totalling 12 fits


2025/11/12 09:56:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 09:56:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ R2=0.7364, RMSE=56015.9136
  ▶ Model: Lasso
Fitting 3 folds for each of 4 candidates, totalling 12 fits


...

    ✓ R2=0.5156, RMSE=75934.5678
  ▶ Model: KNeighbors
Fitting 3 folds for each of 18 candidates, totalling 54 fits


2025/11/12 10:48:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 10:48:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ R2=0.7960, RMSE=49278.2004
  ▶ Model: XGBoost
Fitting 3 folds for each of 144 candidates, totalling 432 fits


2025/11/12 10:50:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 10:50:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ R2=0.8490, RMSE=42396.4797

🎯 All experiments complete! Launch MLflow UI with:
   mlflow ui --port 5000


In [27]:
# Random Forest with no scaler performs best at r2 score of 0.89, so i will be trying different params in detail
    param_grid = {
        "n_estimators": [50, 100,500],
        "max_depth": [None, 20,50],
        "min_samples_split": [2, 5,10],
        "min_samples_leaf": [1, 2,5],
        "max_features": ["sqrt", "log2",None],
        "bootstrap": [True, False],
    }

In [44]:
def random_forest_detailed(df):
    # Split data
    X, y = load_data(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Identify categorical and numeric features
    cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_features = X.select_dtypes(exclude=['object', 'category']).columns.tolist()

    # Preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
        ],
        remainder='passthrough'
    )

    # Model
    rf = RandomForestRegressor(random_state=42)

    # Pipeline
    pipeline = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('model', rf)
    ])

    # Define parameter grid
    param_grid = {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [5, 10, 15, None],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4],
        'model__max_features': ['sqrt', 'log2', None]
    }

    grid = list(ParameterGrid(param_grid))
    print(f"Total runs: {len(grid)}")

    # Loop through every combination and log separately
    for i, params in enumerate(grid):
        with mlflow.start_run(run_name=f"RF_Run_{i+1}"):

            # Set params
            pipeline.set_params(**params)
            pipeline.fit(X_train, y_train)

            # Predictions
            y_pred = pipeline.predict(X_test)

            # Metrics
            test_rmse = rmse(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("rmse", test_rmse)
            mlflow.log_metric("r2_score", r2)

            # Log model
            mlflow.sklearn.log_model(pipeline, name="model")

            print(f"✅ Run {i+1}/{len(grid)} logged | RMSE: {test_rmse:.4f} | R²: {r2:.4f}")




In [45]:
import time
import itertools
from sklearn.model_selection import ParameterGrid

In [46]:
random_forest_detailed(df)

Total runs: 324


2025/11/12 11:28:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run 1/324 logged | RMSE: 53606.7084 | R²: 0.7586


2025/11/12 11:28:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run 2/324 logged | RMSE: 51733.2819 | R²: 0.7751


...

✅ Run 322/324 logged | RMSE: 37819.3845 | R²: 0.8798


2025/11/12 13:29:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run 323/324 logged | RMSE: 38190.5211 | R²: 0.8775


2025/11/12 13:30:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run 324/324 logged | RMSE: 37999.4065 | R²: 0.8787
